In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

# Leitura e Tratamento dos Dados

In [ ]:
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')
display(train)
display(test.head())

In [ ]:
print(f'Train set shape: {train.shape}\nTest set shape: {test.shape}')

In [ ]:
train['Embarked'].value_counts()

In [ ]:
def transforma_embarked(feature):
    if feature == 'S':
        return 0
    elif feature == 'C':
        return 1
    else:
        return 2


train['Embarked_b'] = train['Embarked'].map(transforma_embarked)
test['Embarked_b'] = test['Embarked'].map(transforma_embarked)

In [ ]:
variaveis = ['Sex_b', 'Age', 'Pclass', 'Embarked_b']

In [ ]:
train['Sex'].value_counts()

In [ ]:
def transforma_sexo(feature):
    if feature == 'female':
        return 1
    else:
        return 0


train['Sex_b'] = train['Sex'].map(transforma_sexo)
test['Sex_b'] = test['Sex'].map(transforma_sexo)

In [ ]:
display(train.head())
train['Sex_b'].value_counts()

In [ ]:
X_train = train[variaveis].fillna(-1)
X_test = test[variaveis].fillna(-1)
y_train = train['Survived']

# Decision Tree

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_test)

# Criando o Resultado das Previsões para Importar ao Kaggle

In [ ]:
result = pd.Series(predict, index=test['PassengerId'], name='Survived')
result

In [ ]:
result.to_csv('./yhat/first_decision_tree_model.csv', header=True)